In [10]:
import nltk
from nltk.tokenize import word_tokenize
import random
from nltk import (precision,recall)
import nltk
from nltk.corpus import stopwords
import string
from sklearn.model_selection import train_test_split
import collections

republican = []
democrat = []
with open("presidentsbyparty.txt") as fp:
    names = fp.readlines()

    for x in range(2, names.index('Democrat\n')-1):
            republican.append(names[x].strip())
    for x in range(names.index('Democrat\n'), len(names)):
            democrat.append(names[x].strip())
    

In [11]:
republican_speech = []
democrat_speech = []

speech =''
president_name = ''
with open("sotu.txt") as fp:
    content = fp.readlines()
    for x in content:
        if('***@' in x):
            if(president_name is not ''):
                if(president_name in republican):
                    republican_speech.append(speech)
                if(president_name in democrat):
                    democrat_speech.append(speech)
            x = x[4:]
            president_name = x[0:x.index('@')]
            speech = ''
        else:
            speech = speech + x.replace('\n', ' ')
    if(president_name is not ''):
                if(president_name in republican):
                    republican_speech.append(speech)
                if(president_name in democrat):
                    democrat_speech.append(speech)
#print(republican_speech)
#print(democrat_speech)


In [12]:
documents = []
all_words = []
stop_words = set(stopwords.words('english'))

for x in republican_speech:
    #split into words
    tokens = word_tokenize(x)
    cleaned_tokens = []
    for each_token in tokens:
        #remove puntuation from each word
        table = str.maketrans('', '', string.punctuation)
        token = each_token .translate(table) 
        #remove punctuations from non-aplhabetic words
        #drop stopwords
        if(token not in stop_words and token.isalpha()):
            #convert to lower case
            all_words.append(token.lower())
            cleaned_tokens.append(token)
    documents.append((cleaned_tokens, 'rep'))

for x in democrat_speech:
    #split into words
    tokens = word_tokenize(x)
    cleaned_tokens = []
    for each_token in tokens:
        #remove puntuation from each word
        table = str.maketrans('', '', string.punctuation)
        token = each_token .translate(table) 
        #remove punctuations from non-aplhabetic words
        #drop stopwords
        if( token.isalpha() and token not in stop_words):
            #convert to lower case
            all_words.append(each_token.lower())
            cleaned_tokens = token
    documents.append((cleaned_tokens, 'dem'))

random.shuffle(all_words)
allWords = nltk.FreqDist(all_words)

word_features = list(allWords.keys())[:6000]
#print(word_features)
def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    
    return features


In [13]:
sumAcc = 0
sumPrecRup = 0
sumPreDem = 0
sumRecallRep = 0
sumRecallDem = 0
sumFRep = 0
sumFDem = 0

for i in range(30):
    random.shuffle(documents)
    
    featureSets = [(find_features(speech),cat) for (speech,cat) in documents]
    print(featureSets[0])
    training_set = featureSets[:100]
    test_Set = featureSets[100:]
    
    classifier = nltk.NaiveBayesClassifier.train(training_set)
    print('------------------------')
    print(i)
    
    refsets = collections.defaultdict(set)
    testsets = collections.defaultdict(set)

    for i, (feats, label) in enumerate(test_Set):
        refsets[label].add(i)
        observed = classifier.classify(feats)
        testsets[observed].add(i)
        
    print('Accuracy: ', nltk.classify.accuracy(classifier, test_Set))
    
    print ('Precision for democrats:', nltk.precision(refsets['rep'], testsets['rep']))
    print ('Recall for democrats:', nltk.recall(refsets['rep'], testsets['rep']))
    print ('F-measure for democrats:', nltk.f_measure(refsets['rep'], testsets['rep']))
    
    print ('Precision for republicans:', nltk.precision(refsets['dem'], testsets['dem']))
    print ('Recall for republicans:', nltk.recall(refsets['dem'], testsets['dem']))
    print ('F-measure for republicans:', nltk.f_measure(refsets['dem'], testsets['dem']))
    
    sumAcc = sumAcc + nltk.classify.accuracy(classifier, test_Set)

    sumPrecRup = sumPrecRup + nltk.precision(refsets['rep'], testsets['rep'])
    sumRecallRep = sumRecallRep + nltk.recall(refsets['rep'], testsets['rep'])
    sumFRep = sumFRep + nltk.f_measure(refsets['rep'], testsets['rep'])
    
    sumPreDem = sumPreDem + nltk.precision(refsets['dem'], testsets['dem'])
    sumRecallDem = sumRecallDem + nltk.recall(refsets['dem'], testsets['dem'])
    sumFDem = sumFDem + nltk.f_measure(refsets['dem'], testsets['dem'])
    
print('****************************************************')
print('Average accuracy: ',sumAcc/30)

print('Average precision Republicans: ',sumPrecRup/30)
print('Average recall Republicans: ',sumRecallRep/30)
print('Average F-measure Republicans: ',sumFRep/30)

print('Average precision Democrats: ',sumPreDem/30)
print('Average recall Democrats: ',sumRecallDem/30)
print('Average F-measure Democrats: ',sumFDem/30)


({'sign': True, 'foreign': True, 'delivered': False, 'afflicted': True, 'legislation': True, 'could': True, 'two': True, 'i': False, 'anew': False, 'fail': True, 'hostilities': False, 'congress': False, 'labor': True, 'undeniable': False, 'fact': True, 'arrest': False, 'way': True, 'community': False, 'invite': False, 'great': True, 'asked': False, 'physical': True, 'army': False, 'negotiations': False, 'luxuriant': False, 'constitution': False, 'class': False, 'thus': False, 'certain': True, 'october': False, 'honors': False, 'indict': False, 'effect': False, 'matter': True, 'in': False, 'product': False, 'transaction': False, 'changing': False, 'shall': True, 'troubles': False, 'we': False, 'half': False, 'commissioner': False, 'past': True, 'said': True, 'p': False, 'recovery': False, 'concur': False, 'it': False, 'pipeline': False, 'disarmament': False, 'manufacturers': False, 'missions': False, 'would': True, 'circulation': False, 'consequence': False, 'require': False, 'persons':

KeyboardInterrupt: 